In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import model

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [5]:
import torch
savedmodel = torch.load("saved_model.pt") #, map_location=torch.device('cpu'))

In [10]:
from transformers import DistilBertForSequenceClassification
proppy_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [30]:
dict(savedmodel.distilbert.named_parameters())

{'embeddings.word_embeddings.weight': Parameter containing:
 tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0263],
         [-0.0132, -0.0673, -0.0160,  ..., -0.0227, -0.0554, -0.0260],
         [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0595, -0.0232],
         ...,
         [-0.0231, -0.0588, -0.0105,  ..., -0.0194, -0.0261, -0.0212],
         [-0.0490, -0.0561, -0.0047,  ..., -0.0106, -0.0180, -0.0219],
         [-0.0065, -0.0914, -0.0025,  ..., -0.0151, -0.0504,  0.0460]],
        requires_grad=True),
 'embeddings.position_embeddings.weight': Parameter containing:
 tensor([[ 0.0176, -0.0179, -0.0280,  ..., -0.0037, -0.0092,  0.0186],
         [ 0.0064,  0.0111, -0.0229,  ...,  0.0278,  0.0247, -0.0090],
         [ 0.0005, -0.0059, -0.0075,  ...,  0.0089,  0.0138, -0.0102],
         ...,
         [ 0.0174,  0.0035, -0.0096,  ...,  0.0030,  0.0004, -0.0269],
         [ 0.0217, -0.0060,  0.0147,  ..., -0.0056, -0.0126, -0.0281],
         [ 0.0026, -0.0233,  0.0055, 

In [31]:
dict(proppy_model.distilbert.named_parameters())

{'embeddings.word_embeddings.weight': Parameter containing:
 tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0263],
         [-0.0132, -0.0673, -0.0160,  ..., -0.0227, -0.0554, -0.0260],
         [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0595, -0.0232],
         ...,
         [-0.0231, -0.0588, -0.0105,  ..., -0.0194, -0.0261, -0.0212],
         [-0.0490, -0.0561, -0.0047,  ..., -0.0106, -0.0180, -0.0219],
         [-0.0065, -0.0914, -0.0025,  ..., -0.0151, -0.0504,  0.0460]],
        requires_grad=True),
 'embeddings.position_embeddings.weight': Parameter containing:
 tensor([[ 0.0176, -0.0179, -0.0280,  ..., -0.0037, -0.0092,  0.0186],
         [ 0.0064,  0.0111, -0.0229,  ...,  0.0278,  0.0247, -0.0090],
         [ 0.0005, -0.0059, -0.0075,  ...,  0.0089,  0.0138, -0.0102],
         ...,
         [ 0.0174,  0.0035, -0.0096,  ...,  0.0030,  0.0004, -0.0269],
         [ 0.0217, -0.0060,  0.0147,  ..., -0.0056, -0.0126, -0.0281],
         [ 0.0026, -0.0233,  0.0055, 

In [29]:
source = dict(savedmodel.base_model.named_parameters())
target = dict(proppy_model.base_model.named_parameters())
for name in source:
    target[name].data.copy_(source[name].data)

In [32]:
# Freeze the transferred weights, so we only update the final linear layers in training
for param in proppy_model.base_model.parameters():
    param.requires_grad = False

In [42]:
from dataset import ProppyDataset
from pathlib import Path
# TODO:
GOOGLE_DRIVE_PATH = ""
train_dataset = ProppyDataset(Path(GOOGLE_DRIVE_PATH) / "data_proppy" / "proppy_1.0.train.tsv")
test_dataset = ProppyDataset(Path(GOOGLE_DRIVE_PATH) / "data_proppy" / "proppy_1.0.test.tsv")

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\jense/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\jense/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\jense/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [43]:
from model import model
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    # print(pred.label_ids)
    # print(pred.predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=1e-4,
    weight_decay=0,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

trainer = Trainer(
    model=proppy_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 35986
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 44990
G:\My Drive\487\propaganda_detection\dataset.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}


Epoch,Training Loss,Validation Loss


## Baseline Proppy Distilbert

In [ ]:
baseline_proppy_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    # print(pred.label_ids)
    # print(pred.predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=1e-4,
    weight_decay=0,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

trainer = Trainer(
    model=baseline_proppy_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()